<a href="https://colab.research.google.com/github/Kashara-Alvin-Ssali/Knowledge-Distillation/blob/main/GAT_teacher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import cv2
import numpy as np
from torch_geometric.data import Data
from gat import computeORBGraph, GATClassifier

In [ ]:
# Set paths
trained_model_path = '/content/drive/MyDrive/gat_model.pth'
dataset_train_path = '/content/drive/MyDrive/Dataset4/Training'


In [ ]:
# Load trained GAT teacher
input_dim = 32  # ORB descriptor size (double-check if yours is 32 or 64)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model = GATClassifier(input_dim=input_dim).to(device)
teacher_model.load_state_dict(torch.load(trained_model_path, map_location=device))
teacher_model.eval()

In [1]:
teacher_logits = {}

In [ ]:
# Iterate through all training images
for label_folder in ['Real', 'Fake']:
    folder_path = os.path.join(dataset_train_path, label_folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)

        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        G, keypoints, descriptors = computeORBGraph(image)
        if G is None:
            continue

        edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
        x = torch.tensor(np.array([G.nodes[i]['descriptor'] for i in G.nodes]), dtype=torch.float)

        data = Data(x=x, edge_index=edge_index).to(device)

        with torch.no_grad():
            output = teacher_model(data)  # output shape: (1, 2)

        logits = output.squeeze().cpu()  # shape: (2,)
        teacher_logits[img_path] = logits


In [ ]:
# Save teacher logits to file
torch.save(teacher_logits, '/content/drive/MyDrive/teacher_logits.pt')
print("Teacher logits saved successfully!")